In [13]:
!pip install pandas
!pip install openpyxl
!pip install yfiles_jupyter_graphs
!pip install networkx



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [15]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")

df = pd.read_excel(file_path)
print(df)

                                         Unnamed: 1           Epoch  \
0               NaN                             NaN           Cycle   
1               NaN                             NaN  Cycle End Rule   
2               NaN                             NaN     Visit Label   
3               NaN                             NaN       Condition   
4               NaN                             NaN    Visit Window   
5   Parent Activity                  Child Activity             BCs   
6               NaN                Informed Consent             NaN   
7               NaN  Inclusion / Exclusion Criteria             NaN   
8               NaN            Physical Examination            PE 1   
9               NaN                             NaN            PE 2   
10              NaN                             NaN          Weight   
11              NaN                             NaN          Height   

        Screening Treatment   Unnamed: 5  Unnamed: 6   Unnamed: 7  Unnamed: 

In [16]:

# shows empty widget
w = GraphWidget()
w.set_nodes([
            {'id': 0, 'properties': {'label': 'Entry'}},
            {'id': 1, 'properties': {'label': 'Exit'}}
        ])
w.set_edges([
  {id: 0, 'start': 0, 'end': 1, 'properties': {'label': "complete"}}
])
w.orthogonal_layout()
w

/Users/daveih/Library/Python/3.10/lib/python/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not builtin_function_or_method
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


GraphWidget(layout=Layout(height='500px', width='100%'))

In [17]:
for (colname,colval) in df.iteritems():
    print(colname, colval.values)

  [nan nan nan nan nan 'Parent Activity' nan nan nan nan nan nan]
Unnamed: 1 [nan nan nan nan nan 'Child Activity' 'Informed Consent'
 'Inclusion / Exclusion Criteria' 'Physical Examination' nan nan nan]
Epoch ['Cycle' 'Cycle End Rule' 'Visit Label' 'Condition' 'Visit Window' 'BCs'
 nan nan 'PE 1' 'PE 2' 'Weight' 'Height']
Screening [nan nan nan 'N: 0..30 Days' nan nan 'X' 'X' 'X' nan nan nan]
Treatment [1 nan 'Day 1' 'A:' nan nan nan 'X' 'X' nan nan nan]
Unnamed: 5 [nan nan 'Day 15' 'P: +14 Days' '-2..2 Days' nan nan nan 'X' nan nan nan]
Unnamed: 6 [2 nan 'Day 1' 'P: +N Days' '-2..2 Days' nan nan nan 'X' nan nan nan]
Unnamed: 7 [nan nan 'Day 15' 'P: +14 Days' '-2..2 Days' nan nan nan 'X' nan nan nan]
Unnamed: 8 [3 nan 'Day 1' 'P: +N Days' '-2..2 Days' nan nan nan 'X' nan nan nan]
Unnamed: 9 [nan nan 'Day 15' 'P: +14 Days' '-2..2 Days' nan nan nan nan nan nan nan]
Unnamed: 10 ['4..12' nan 'Day 1' 'P: +N Days' '-2..2 Days' nan nan nan 'X' nan nan nan]
Unnamed: 11 ['13..N step 3' nan 'Da

/var/folders/qw/xmdzmhzj3217xnnh7xz437wm0000gn/T/ipykernel_98104/3658610266.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for (colname,colval) in df.iteritems():


In [21]:
for (index, colname) in enumerate(df):
    if index > 2:
      print(index, df[colname].values[2])

3 nan
4 Day 1
5 Day 15
6 Day 1
7 Day 15
8 Day 1
9 Day 15
10 Day 1
11 Day 1
12 EOT Visit
13 nan
